In [1]:
'Hi iam trying to push into github repo'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load the CSV file
file_path = "soren.csv"  # Ensure this path is correct
data = pd.read_csv(file_path)

# Drop unnecessary columns
data.drop(['Frame', 'Timestamp', 'Average Pressure (mmHg)', 'Minimum Pressure (mmHg)', 
           'Maximum Pressure (mmHg)', 'Standard Pressure Deviation (mmHg)', 
           'Median Pressure (mmHg)', 'Contact Area (m²)', 'Total Area (m²)', 
           'Estimated Force (N)', 'Range Min (mmHg)', 'Range Max (mmHg)'], axis=1, inplace=True)

# Remove rows with all zero values
data = data[(data != 0).any(axis=1)]

# Create a directory to save the images
output_folder = "images_32x32"
os.makedirs(output_folder, exist_ok=True)

# Get the number of rows after filtering
num_rows = data.shape[0]

# Loop through each row in the dataset and save it as a 32x32 image
for i in range(num_rows):
    # Reshape the data into a 32x32 grid
    grid_data = data.iloc[i].values.reshape(32, 32)
    
    # Create a plot without axis
    plt.figure(figsize=(6, 6))
    plt.imshow(grid_data, cmap='viridis', interpolation='none')
    plt.axis('off')  # Remove axis
    
    # Save the image in the output folder
    image_filename = os.path.join(output_folder, f"pressure_image_{i+1}.png")
    plt.savefig(image_filename, bbox_inches='tight', pad_inches=0)
    plt.close()

print(f"Saved {num_rows} images in the '{output_folder}' folder after removing rows with all zero values.")


Saved 510 images in the 'images_32x32' folder after removing rows with all zero values.


In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load the CSV file
file_path = "soren.csv"  # Ensure this path is correct
data = pd.read_csv(file_path)

# Drop unnecessary columns
columns_to_drop = ['Frame', 'Timestamp', 'Average Pressure (mmHg)', 'Minimum Pressure (mmHg)', 
                   'Maximum Pressure (mmHg)', 'Standard Pressure Deviation (mmHg)', 
                   'Median Pressure (mmHg)', 'Contact Area (m²)', 'Total Area (m²)', 
                   'Estimated Force (N)', 'Range Min (mmHg)', 'Range Max (mmHg)']

# Drop metadata columns if they exist
data.drop(columns=[col for col in columns_to_drop if col in data.columns], axis=1, inplace=True)

# Remove rows where all values are zero
data = data[(data != 0).any(axis=1)]

# Create a directory to save the 6x6 images
output_folder_6x6 = "images_6x6"
os.makedirs(output_folder_6x6, exist_ok=True)

# Get the number of rows after filtering
num_rows = data.shape[0]

# Generate 6 evenly spaced indices from a 32×32 grid
indices = np.linspace(2, 30, 6, dtype=int)  # Choosing 6 evenly spaced points in a 32x32 grid

# Generate (x, y) coordinate pairs for the 6×6 grid
selected_indices = [(indices[j], indices[k]) for j in range(6) for k in range(6)]

# Loop through each row in the dataset and save it as a 6x6 image
for i in range(num_rows):
    # Reshape the data into a 32x32 grid
    grid_data = data.iloc[i].values.reshape(32, 32)
    
    # Extract the 6x6 grid correctly
    reduced_grid_data = np.array([
        [grid_data[x, y] for y in np.linspace(2, 30, 6, dtype=int)]  # Select columns
        for x in np.linspace(2, 30, 6, dtype=int)  # Select rows
    ])
    
    # Create a plot without axis
    plt.figure(figsize=(4, 4))
    plt.imshow(reduced_grid_data, cmap='viridis', interpolation='none')
    plt.axis('off')  # Remove axis
    
    # Save the image in the output folder
    image_filename_6x6 = os.path.join(output_folder_6x6, f"pressure_image_6x6_{i+1}.png")
    plt.savefig(image_filename_6x6, bbox_inches='tight', pad_inches=0)
    plt.close()

print(f"Saved {num_rows} 6x6 images in the '{output_folder_6x6}' folder after processing the original data.")


Saved 510 6x6 images in the 'images_6x6' folder after processing the original data.


In [3]:
# Load the 290th 32×32 grid
grid_data = data.iloc[500].values.reshape(32, 32)  # Adjust index (290th image)

# Extract the 6×6 grid using the same method
row_indices = np.linspace(3, 28, 6, dtype=int)  # Ensure evenly spaced
col_indices = np.linspace(3, 28, 6, dtype=int)
extracted_6x6 = np.array([[grid_data[x, y] for y in col_indices] for x in row_indices])

# Print extracted 6x6 values
print("Extracted 6×6 Grid:")
print(extracted_6x6)

# Print corresponding values in the original 32×32 grid
print("\nOriginal 32×32 Grid (Subset of Selected Indices):")
for x in row_indices:
    print([grid_data[x, y] for y in col_indices])


Extracted 6×6 Grid:
[[ 0  0  0 11 26 40]
 [ 0  0 32 42 22 10]
 [ 0  0  6  3  0  0]
 [ 0  0  5 11  5  3]
 [ 0  0  0  2  0  0]
 [ 0  0  0  0  0  0]]

Original 32×32 Grid (Subset of Selected Indices):
[np.int64(0), np.int64(0), np.int64(0), np.int64(11), np.int64(26), np.int64(40)]
[np.int64(0), np.int64(0), np.int64(32), np.int64(42), np.int64(22), np.int64(10)]
[np.int64(0), np.int64(0), np.int64(6), np.int64(3), np.int64(0), np.int64(0)]
[np.int64(0), np.int64(0), np.int64(5), np.int64(11), np.int64(5), np.int64(3)]
[np.int64(0), np.int64(0), np.int64(0), np.int64(2), np.int64(0), np.int64(0)]
[np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]


In [7]:
import numpy as np
import pandas as pd

# Define row and column indices for 6×6 selection
row_indices = np.linspace(3, 28, 6, dtype=int)  # 6 evenly spaced rows
col_indices = np.linspace(3, 28, 6, dtype=int)  # 6 evenly spaced columns

# List to store extracted grids for all images
extracted_grids = []

# Loop through each image (row) in the dataset
for i in range(len(data)):  
    grid_data = data.iloc[i].values.reshape(32, 32)  # Reshape to 32×32 grid
    extracted_6x6 = np.array([[grid_data[x, y] for y in col_indices] for x in row_indices])  # Extract 6x6 grid
    extracted_grids.append(extracted_6x6.flatten())  # Flatten 6x6 grid into a row

# Convert list to DataFrame
extracted_df = pd.DataFrame(extracted_grids)

# Save to CSV file
extracted_df.to_csv("extracted_6x6_grids.csv", index=False)

print("CSV file 'extracted_6x6_grids.csv' has been saved successfully!")


NameError: name 'data' is not defined

In [5]:
import numpy as np
import pandas as pd

# Load the extracted CSV
extracted_df = pd.read_csv("extracted_6x6_grids.csv")

# Choose a random image index to check (e.g., 290th image)
image_index = 500  # Adjust to match an index in your dataset

# Get the original 32x32 grid
grid_data = data.iloc[image_index].values.reshape(32, 32)

# Define row and column indices used for extraction
row_indices = np.linspace(3, 28, 6, dtype=int)
col_indices = np.linspace(3, 28, 6, dtype=int)

# Extract 6x6 grid manually again
manual_extracted_6x6 = np.array([[grid_data[x, y] for y in col_indices] for x in row_indices])

# Load the extracted row from CSV and reshape to 6×6 for comparison
extracted_6x6_from_csv = extracted_df.iloc[image_index].values.reshape(6, 6)

# Print the original extracted 6x6 grid
print("Original Extracted 6×6 Grid:")
print(manual_extracted_6x6)

# Print the saved extracted 6x6 grid from the CSV
print("\nExtracted 6×6 Grid from CSV:")
print(extracted_6x6_from_csv)

# Check if they are the same
if np.array_equal(manual_extracted_6x6, extracted_6x6_from_csv):
    print("\n✅ The extracted values match! The CSV is correct.")
else:
    print("\n❌ Mismatch detected! Check extraction logic.")


Original Extracted 6×6 Grid:
[[ 0  0  0 11 26 40]
 [ 0  0 32 42 22 10]
 [ 0  0  6  3  0  0]
 [ 0  0  5 11  5  3]
 [ 0  0  0  2  0  0]
 [ 0  0  0  0  0  0]]

Extracted 6×6 Grid from CSV:
[[ 0  0  0 11 26 40]
 [ 0  0 32 42 22 10]
 [ 0  0  6  3  0  0]
 [ 0  0  5 11  5  3]
 [ 0  0  0  2  0  0]
 [ 0  0  0  0  0  0]]

✅ The extracted values match! The CSV is correct.


In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.models import vgg16, VGG16_Weights
from PIL import Image
import torch.nn.functional as F

# ===========================
# Device Setup
# ===========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ===========================
# Generator Model (6×6 → 32×32)
# ===========================
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # Use ConvTranspose2d-based upsampling for learned upscaling.
        self.model = nn.Sequential(
            # Input: 6×6 → remains 6×6
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            # Upsample: 6×6 → 12×12
            nn.ConvTranspose2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            
            # Upsample: 12×12 → 24×24
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            
            # Upsample: 24×24 → 32×32 (Here we use kernel_size=4, stride=2, padding=1 would give 48×48,
            # so we change it to a smaller scaling factor. One way is to use an interpolation here.)
            # Option 1: Use ConvTranspose2d with modified parameters:
            # nn.ConvTranspose2d(64, 3, kernel_size=3, stride=1, padding=1),
            # Option 2: Use Upsample with bilinear interpolation.
            nn.Upsample(scale_factor=4/3, mode="bilinear", align_corners=True),  # 24×24 → 32×32
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

# ===========================
# Discriminator Model (for 32×32 images)
# ===========================
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(512, 1, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# ===========================
# Initialize Models
# ===========================
generator = Generator().to(device)
discriminator = Discriminator().to(device)
print("✅ Generator & Discriminator initialized.")

# ===========================
# Loss Functions & Optimizers
# ===========================
criterion = nn.BCELoss()  # Adversarial loss
optimizer_G = optim.Adam(generator.parameters(), lr=5e-5, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=5e-5, betas=(0.5, 0.999))

# ===========================
# Perceptual Loss (VGG16-based)
# ===========================
class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        vgg = vgg16(weights=VGG16_Weights.IMAGENET1K_V1).features[:16]
        for param in vgg.parameters():
            param.requires_grad = False
        self.vgg = vgg.to(device)
        self.resize = nn.Upsample(size=(224, 224), mode='bilinear', align_corners=True)

    def forward(self, generated, target):
        # Resize both generated and target to 224x224
        gen_resized = self.resize(generated)
        target_resized = self.resize(target)
        return nn.functional.mse_loss(self.vgg(gen_resized), self.vgg(target_resized))

perceptual_loss = PerceptualLoss()

# ===========================
# (Optional) Edge Loss to Improve Sharpness
def edge_loss(generated, target):
    # Create a Sobel filter for one channel
    sobel_x = torch.tensor([[1, 0, -1],
                             [2, 0, -2],
                             [1, 0, -1]], dtype=torch.float32)
    sobel_y = torch.tensor([[1, 2, 1],
                             [0, 0, 0],
                             [-1, -2, -1]], dtype=torch.float32)
    
    # Reshape to [1, 1, 3, 3] and repeat for 3 channels
    sobel_x = sobel_x.unsqueeze(0).unsqueeze(0).to(generated.device)  # shape [1,1,3,3]
    sobel_y = sobel_y.unsqueeze(0).unsqueeze(0).to(generated.device)
    
    # Repeat filters along the channel dimension for each input channel
    sobel_x = sobel_x.repeat(3, 1, 1, 1)  # shape becomes [3,1,3,3]
    sobel_y = sobel_y.repeat(3, 1, 1, 1)
    
    # Apply convolution using groups=3 to apply the filter separately for each channel
    gen_edges_x = nn.functional.conv2d(generated, sobel_x, padding=1, groups=3)
    gen_edges_y = nn.functional.conv2d(generated, sobel_y, padding=1, groups=3)
    target_edges_x = nn.functional.conv2d(target, sobel_x, padding=1, groups=3)
    target_edges_y = nn.functional.conv2d(target, sobel_y, padding=1, groups=3)
    
    # Use L1 loss for edge differences
    return nn.functional.l1_loss(gen_edges_x, target_edges_x) + nn.functional.l1_loss(gen_edges_y, target_edges_y)


# ===========================
# Data Preparation Functions
# ===========================
# For low-resolution images (Generator Input): Resize to 6x6
def prepare_low_res(folder_path):
    images = []
    transform = transforms.Compose([
        transforms.Resize((6, 6)),  # Force low-res images to 6x6
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            img = Image.open(os.path.join(folder_path, filename)).convert("RGB")
            img = transform(img)
            images.append(img)
    return torch.stack(images)

# For high-resolution images (Target): Resize to 32x32
def prepare_high_res(folder_path):
    images = []
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # Target is 32x32
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            img = Image.open(os.path.join(folder_path, filename)).convert("RGB")
            img = transform(img)
            images.append(img)
    return torch.stack(images)

# Load Data
low_res_images = prepare_low_res('images_6x6').to(device)
high_res_images = prepare_high_res('images_32x32').to(device)
print(f"Low-res images shape: {low_res_images.shape}")
print(f"High-res images shape: {high_res_images.shape}")

# ===========================
# Training Loop
# ===========================
num_epochs = 250
batch_size = 4

for epoch in range(num_epochs):
    for i in range(0, len(low_res_images), batch_size):
        low_res = low_res_images[i:i+batch_size].to(device)   # Expected shape: [B, 3, 6, 6]
        high_res = high_res_images[i:i+batch_size].to(device)  # Expected shape: [B, 3, 32, 32]

        # Train Discriminator
        optimizer_D.zero_grad()
        real_output = discriminator(high_res).view(-1)
        d_loss_real = criterion(real_output, torch.ones_like(real_output).to(device))
        
        fake_image = generator(low_res)
        fake_output = discriminator(fake_image.detach()).view(-1)
        d_loss_fake = criterion(fake_output, torch.zeros_like(fake_output).to(device))
        
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_D.step()

        # Train Generator (2 updates per discriminator update)
        for _ in range(2):
            optimizer_G.zero_grad()
            fake_image = generator(low_res)
            fake_output = discriminator(fake_image).view(-1)
            g_bce_loss = criterion(fake_output, torch.ones_like(fake_output).to(device))
            g_perceptual_loss = perceptual_loss(fake_image, high_res)
            g_edge_loss = edge_loss(fake_image, high_res)
            g_loss = g_bce_loss + 0.3 * g_perceptual_loss + 0.1 * g_edge_loss
            g_loss.backward()
            optimizer_G.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    # Save checkpoint every 50 epochs
    if (epoch + 1) % 50 == 0:
        os.makedirs("models", exist_ok=True)
        torch.save(generator.state_dict(), f"models/generator_epoch{epoch+1}.pth")
        torch.save(discriminator.state_dict(), f"models/discriminator_epoch{epoch+1}.pth")
        print(f"✅ Saved checkpoint at epoch {epoch+1}")

print("✅ Training Complete! Saving Final Models...")
torch.save(generator.state_dict(), "models/generator.pth")
torch.save(discriminator.state_dict(), "models/discriminator.pth")
print("✅ Models Saved Successfully!")


Using device: cpu
✅ Generator & Discriminator initialized.
Low-res images shape: torch.Size([510, 3, 6, 6])
High-res images shape: torch.Size([510, 3, 32, 32])
Epoch [1/250], D Loss: 0.7568, G Loss: 1.9224
Epoch [2/250], D Loss: 0.2392, G Loss: 3.0120
Epoch [3/250], D Loss: 0.3755, G Loss: 2.8876
Epoch [4/250], D Loss: 4.2517, G Loss: 0.9073
Epoch [5/250], D Loss: 0.2801, G Loss: 2.4069
Epoch [6/250], D Loss: 1.2545, G Loss: 1.0309
Epoch [7/250], D Loss: 0.9426, G Loss: 1.1877
Epoch [8/250], D Loss: 1.0315, G Loss: 1.2157
Epoch [9/250], D Loss: 1.1781, G Loss: 1.0051
Epoch [10/250], D Loss: 1.2780, G Loss: 0.8916
Epoch [11/250], D Loss: 1.3572, G Loss: 0.8356
Epoch [12/250], D Loss: 1.3438, G Loss: 0.8435
Epoch [13/250], D Loss: 1.3243, G Loss: 0.7921
Epoch [14/250], D Loss: 1.3045, G Loss: 0.7798
Epoch [15/250], D Loss: 1.2894, G Loss: 0.7628
Epoch [16/250], D Loss: 1.2787, G Loss: 0.8261
Epoch [17/250], D Loss: 1.2751, G Loss: 0.9118
Epoch [18/250], D Loss: 1.2640, G Loss: 0.9101
Epo

In [10]:
import os
import torch
from PIL import Image
import torchvision.transforms as transforms

# Make sure to define or import your Generator class before using it
# from your_model_file import Generator

# Load Trained Generator
generator = Generator()
generator.load_state_dict(torch.load("models/generator.pth"))
generator.eval()

# Define folders
input_folder = "images_6x6"
output_folder = "upscaled_250_32x32"
os.makedirs(output_folder, exist_ok=True)

# Correct Transformations for low-res image
transform = transforms.Compose([
    transforms.Resize((6, 6)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Match training normalization
])

# Process images and save with new file names
for i, filename in enumerate(os.listdir(input_folder), start=1):
    if filename.endswith(".png"):
        input_path = os.path.join(input_folder, filename)

        # Load and preprocess the image
        test_image = Image.open(input_path).convert("RGB")
        low_res_input = transform(test_image).unsqueeze(0)  # Add batch dimension

        # Generate upscaled 32×32 image
        with torch.no_grad():
            upscaled_image = generator(low_res_input).squeeze(0)  # Remove batch dimension

            # Convert from [-1, 1] to [0, 1]
            upscaled_image = (upscaled_image + 1) / 2

            # Convert tensor to PIL image
            upscaled_image = transforms.ToPILImage()(upscaled_image)
            
            # Create the new file name following the pattern
            new_filename = f"pressure_image_32x32_{i}.png"
            output_path = os.path.join(output_folder, new_filename)
            
            # Save the upscaled image
            upscaled_image.save(output_path)

        print(f"✅ Upscaled and saved: {output_path}")

print("🚀 All images successfully upscaled to upscaled_250_32×32 and saved!")


NameError: name 'Generator' is not defined

In [11]:
# Function to visualize images
def visualize_images(low_res, generated_high_res, original_high_res, psnr_value, rmse_value):
    import matplotlib.pyplot as plt

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    axs[0].imshow(low_res.permute(1, 2, 0).numpy())
    axs[0].set_title('Low Resolution Image')
    axs[0].axis('off')

    axs[1].imshow(generated_high_res.permute(1, 2, 0).detach().numpy())
    axs[1].set_title(f'Generated High Res\nPSNR: {psnr_value:.2f}, RMSE: {rmse_value:.2f}')
    axs[1].axis('off')

    axs[2].imshow(original_high_res.permute(1, 2, 0).numpy())
    axs[2].set_title('Original High Res Image')
    axs[2].axis('off')

    plt.tight_layout()
    plt.show()

# Function to evaluate and display images
def evaluate_image(index):
    # Ensure index is within bounds
    if index < 0 or index >= len(low_res_images):
        print("Index out of range. Please select a valid index.")
        return

    # Get the images
    low_res = low_res_images[index].unsqueeze(0)  # Add batch dimension
    original_high_res = high_res_images[index]

    # Generate high-resolution image
    generator.eval()  # Set generator to evaluation mode
    with torch.no_grad():
        generated_high_res = generator(low_res).squeeze(0)  # Remove batch dimension

    # Calculate PSNR and RMSE
    psnr_value = psnr(original_high_res.numpy(), generated_high_res.numpy())
    rmse_value = np.sqrt(mse(original_high_res.numpy(), generated_high_res.numpy()))

    # Visualize images and metrics
    visualize_images(low_res.squeeze(0), generated_high_res, original_high_res, psnr_value, rmse_value)

# Example usage
evaluate_image(460)  # Replace 0 with the desired index


NameError: name 'low_res_images' is not defined

In [ ]:
'''import os
import numpy as np
import torch
from PIL import Image

# Function to save the generated high-res images
def save_generated_images(output_folder):
    # Create the folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all low-res images
    generator.eval()  # Set generator to evaluation mode
    for i, low_res in enumerate(low_res_images):
        # Add batch dimension
        low_res = low_res.unsqueeze(0)

        # Generate high-resolution image
        with torch.no_grad():
            generated_high_res = generator(low_res).squeeze(0)  # Remove batch dimension

        # Convert to NumPy array and scale to 0-255
        generated_image = (generated_high_res.permute(1, 2, 0).detach().numpy() * 255).astype(np.uint8)

        # Convert to PIL image for saving
        pil_image = Image.fromarray(generated_image)

        # Save the image
        image_path = os.path.join(output_folder, f'gen_img_{i+1}.png')
        pil_image.save(image_path)

        print(f"Saved: {image_path}")

# Example usage
output_folder = "generated_6"
save_generated_images(output_folder)'''


In [2]:
# Function to evaluate PSNR and RMSE for all images
def evaluate_all_images():
    psnr_values = []
    rmse_values = []

    generator.eval()  # Set generator to evaluation mode
    for i in range(len(low_res_images)):
        low_res = low_res_images[i].unsqueeze(0)  # Add batch dimension
        original_high_res = high_res_images[i]

        # Generate high-resolution image
        with torch.no_grad():
            generated_high_res = generator(low_res).squeeze(0)  # Remove batch dimension

        # Calculate PSNR and RMSE
        psnr_value = psnr(original_high_res.numpy(), generated_high_res.numpy())
        rmse_value = np.sqrt(mse(original_high_res.numpy(), generated_high_res.numpy()))

        # Append metrics to the lists
        psnr_values.append(psnr_value)
        rmse_values.append(rmse_value)

    return psnr_values, rmse_values

# Evaluate metrics for all images
psnr_values_all, rmse_values_all = evaluate_all_images()

import matplotlib.pyplot as plt

# Function to plot PSNR and RMSE with enhanced inbuilt style
def plot_metrics_with_enhanced_style(psnr_values, rmse_values):
    # Use a professional built-in style
    plt.style.use('fivethirtyeight')  # A clean, visually pleasing style

    indices = range(len(psnr_values))  # Image indices

    # Plot PSNR values
    plt.figure(figsize=(14, 7))
    plt.plot(
        indices, psnr_values, label='PSNR (dB)', color='#007acc',
        marker='o', markersize=5, linewidth=2, alpha=0.9
    )
    plt.fill_between(indices, psnr_values, color='#007acc', alpha=0.2)  # Add shading for emphasis
    plt.xlabel('Image Index', fontsize=14, fontweight='bold')
    plt.ylabel('PSNR (dB)', fontsize=14, fontweight='bold')
    plt.title('PSNR vs. Image Index', fontsize=18, fontweight='bold')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(fontsize=12, loc='upper right', frameon=True, shadow=True)
    plt.tight_layout()
    plt.show()

    # Plot RMSE values
    plt.figure(figsize=(14, 7))
    plt.plot(
        indices, rmse_values, label='RMSE', color='#ff5733',
        marker='s', markersize=5, linewidth=2, alpha=0.9
    )
    plt.fill_between(indices, rmse_values, color='#ff5733', alpha=0.2)  # Add shading for emphasis
    plt.xlabel('Image Index', fontsize=14, fontweight='bold')
    plt.ylabel('RMSE', fontsize=14, fontweight='bold')
    plt.title('RMSE vs. Image Index', fontsize=18, fontweight='bold')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(fontsize=12, loc='upper right', frameon=True, shadow=True)
    plt.tight_layout()
    plt.show()

# Example usage
plot_metrics_with_enhanced_style(psnr_values_all, rmse_values_all)


NameError: name 'generator' is not defined